In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import os
import torch
import re
import string
import json

import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import torch.nn as nn
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import AutoModel


In [ ]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from itertools import combinations
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('/kaggle/input/clean-data/cleandata.csv')

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df, test_size=0.15, random_state=42)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 4e-7
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
target_cols = [col for col in df.columns if col not in ['Text','Title','Abstract']]
len(target_cols)

In [ ]:
train_dataset = BERTDataset(df_train, tokenizer, MAX_LEN)
test_dataset = BERTDataset(df_test, tokenizer, MAX_LEN)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=4, shuffle=False, pin_memory=True)

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel
from torch.cuda.amp import autocast, GradScaler

class MultiLabelSciBERT(nn.Module):
    def __init__(self, num_labels, hidden_dim=768, lstm_hidden=384, num_layers=2, bidirectional=True):
        super(MultiLabelSciBERT, self).__init__()
        self.sci_bert = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

        # Optionally, freeze SciBERT parameters
        for param in self.sci_bert.parameters():
            param.requires_grad = False

        self.bilstm = nn.LSTM(input_size=hidden_dim, 
                              hidden_size=lstm_hidden, 
                              num_layers=num_layers, 
                              bidirectional=bidirectional, 
                              batch_first=True)
        
        lstm_output_features = lstm_hidden * 2 if bidirectional else lstm_hidden
        self.classifier = nn.Linear(lstm_output_features, num_labels)

    def forward(self, ids, mask):
        outputs = self.sci_bert(ids, attention_mask=mask)
        sequence_output = outputs[0]
        
        lstm_output, _ = self.bilstm(sequence_output)
        pooled_output = lstm_output[:, 0, :]  # Use the output of the first token for classification
        
        logits = self.classifier(pooled_output)
        return logits


In [ ]:
def train(model, train_loader, optimizer, loss_function, device, epoch, scaler):
    model.train()
    for _, data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)

        optimizer.zero_grad()

        with autocast():
            logits = model(ids, mask)
            loss = loss_function(logits, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if _ % 500 == 0:
            print(f'Epoch: {epoch}, Loss: {loss.item()}')


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 57  # Update this with the actual number of labels for your task
model = MultiLabelSciBERT(num_labels=num_labels).to(device)



In [ ]:
# Before loading custom state, check some SciBERT layer weights
print("Before loading custom checkpoint:")
for name, param in model.named_parameters():
    if 'sci_bert' in name and 'embeddings' in name:
        print(f"{name}: {param.mean().item()}")

# Assuming you've loaded your model as shown previously
custom_pretrained_state = torch.load('/kaggle/input/model-new/model5.pth', map_location=device)
model.load_state_dict(custom_pretrained_state, strict=False)

# After loading custom state, check the same SciBERT layer weights
print("\nAfter loading custom checkpoint:")
for name, param in model.named_parameters():
    if 'sci_bert' in name and 'embeddings' in name:
        print(f"{name}: {param.mean().item()}")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DistributionBalancedLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, beta=0.690, epsilon=1e-9, max_factor=40.0):
        super(DistributionBalancedLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.beta = beta
        self.epsilon = epsilon
        self.max_factor = max_factor  # Maximum allowed value for class_balancing_factor

    def forward(self, logits, targets):
        # logits: [N, C], targets: [N, C] where N is batch size and C is number of classes
        probs = torch.sigmoid(logits)
        pt = probs * targets + (1 - probs) * (1 - targets)  # pt is the probability of the prediction being correct

        # Class balancing factor
        class_freq = torch.sum(targets, dim=0) / (targets.size(0) + self.epsilon)  # Added epsilon to avoid division by zero
        class_balancing_factor = (1 - self.beta) / (1 - torch.pow(self.beta, class_freq))

        # Cap the class_balancing_factor to avoid it becoming too large
        class_balancing_factor = torch.clamp(class_balancing_factor, max=self.max_factor)

        # Modulating factor
        modulating_factor = torch.pow(1.0 - pt, self.gamma)

        # Focal loss component
        focal_loss = -self.alpha * modulating_factor * torch.log(pt + self.epsilon)

        # Applying class balancing
        balanced_focal_loss = class_balancing_factor * focal_loss

        # Average over the batch
        loss = torch.mean(torch.sum(balanced_focal_loss, dim=1))

        return loss


In [ ]:
loss_function = DistributionBalancedLoss().to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)


scaler = GradScaler()

EPOCHS = 15  # Adjust based on your needs
# Assume train_loader is defined elsewhere

for epoch in range(EPOCHS):
    train(model, train_loader, optimizer, loss_function, device, epoch, scaler)

In [ ]:
torch.save(model.state_dict(), 'model_kriti2.pth')


In [ ]:
def validation():
    model.eval() 
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(test_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask)
#              outputs = model(ids, mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation()
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

In [ ]:
import pandas as pd
# Load the test dataset
test_df = pd.read_csv("/kaggle/input/kriti-csv/test.csv")

# Preprocess the text data
test_df['Text'] = test_df['Title'] + ' ' + test_df['Abstract']
test_df = test_df.drop(['Title', 'Abstract'], axis=1)
test_df['Text']=test_df['Text'].str.lower()


In [ ]:
def remove_punct(text):
    punctuation =string.punctuation
    return text.translate(str.maketrans('' , '',punctuation))
test_df['Text']=test_df['Text'].apply(lambda x: remove_punct(x))

In [ ]:
from nltk.corpus import stopwords 
",".join(stopwords.words('english'))
STOPWRDS=set(stopwords.words('english'))

In [ ]:
def remove_stp(Text):
    return " ".join([word for word in Text.split() if word not in STOPWRDS])

In [ ]:
test_df['Text']=test_df['Text'].apply(lambda x: remove_stp(x))

In [ ]:
not_needed = [
     "using", "results", "performance", "show", "method", "approach",
    "also", "based", "problem", "proposed", "two", "information", "new", "large",
    "different", "study", "however",  "analysis", "one", "work", "used", "first",
    "set", "use", "existing", "present", "process", "demonstrate", "task", "general",
    "several", "due", "compared", "via", "moreover", "eg", "thus", "possible", "make",
    "like", "important", "key", "furthermore", "give", "state", "certain", "ie", "form",
    "allows", "finally", "often", "even", "many", "various", "well", "also", "however",
    "several", "due", "across", "may", "without", "among", "including", "particular",
    "especially", "either", "often", "even", "moreover", "thus", "ie", "eg", "although",
    "despite","would" ,"within" ,"",
]

import pandas as pd
def not_need(Text):
    return " ".join([word for word in Text.split() if word not in not_needed])

test_df['Text'] = test_df['Text'].apply(lambda x:not_need(x))

In [ ]:
class BERTTestDatasets(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

# Create the test dataset
result_dataset = BERTTestDatasets(test_df, tokenizer, MAX_LEN)


In [ ]:
result_loader = DataLoader(result_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=4)


In [ ]:
def predict():
    model.eval()
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(result_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model(ids, mask)
            predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return predictions

# Get predictions
test_predictions = predict()


In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to binary (True/False)
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols)
submission_df.insert(0, 'Id', test_df.Id)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
target_cols2 = [col for col in df.columns if col not in ['Text','Title','Abstract']]
len(target_cols2)

In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to binary (True/False)
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols2)
submission_df.insert(0, 'Id', test_df.Id)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
template_df = pd.read_csv('/kaggle/input/kriti-csv/sample_submission.csv')
required_column_order = template_df.columns.tolist()


In [ ]:
# Ensure that 'Id' is the first column
required_column_order.remove('Id')
required_column_order.insert(0, 'Id')

# Reorder your DataFrame columns
submission_df = submission_df[required_column_order]


In [ ]:
submission_df
submission_df.to_csv('/kaggle/working/submission_ordered_wow.csv', index=False)


In [ ]:
submission_df